Client code for Dropbox's API

In [70]:
import webbrowser
import http.server
import socketserver
from urllib import request
from urllib.request import urlopen
from urllib.parse import urlencode
import urllib
import http.server
import socket
import re
import time
import json

In [2]:
f = open("app_info.txt", "r") ## Contains private info about application
APP_KEY = f.readline().rstrip().replace("app key ", "")
APP_SECRET = f.readline().rstrip().replace("secret ", "")
# Generated token so that you don't have to run server each time when testing. Comment out if running server
# token = f.readline().rstrip().replace("token ", "") 

In [67]:
# # Server code (can skip for now)
authorize_url =("https://www.dropbox.com/oauth2/authorize?client_id=" + APP_KEY + 
      "&response_type=code&redirect_uri=http://localhost:8000")
webbrowser.open(authorize_url)

HOST, PORT = "localhost", 8000
data = "unecessary"

received = None
while (received == None or (isinstance(received,str) and len(received) < 3) or
    (isinstance(received,bytes) and "code" not in received.decode("utf-8"))):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        # Connect to server and send data
        sock.connect((HOST, PORT))
        print(sock.getsockname())


        print("Sent: {}".format(data))
        sock.sendall(bytes(data + "\n", "utf-8"))
        # Receive data from the server and shut down

        received = str(sock.recv(1024), "utf-8")
        print("Received: {}".format(received[0:10]))
    time.sleep(2)
code = re.search("(?<=code=)(.*?)(?=[!\s])", received).group()

('127.0.0.1', 59515)
Sent: unecessary
Received: 
('127.0.0.1', 59517)
Sent: unecessary
Received: 
('127.0.0.1', 59518)
Sent: unecessary
Received: 
('127.0.0.1', 59523)
Sent: unecessary
Received: GET /?code


In [68]:
head = {"code": code,
       "grant_type": "authorization_code",
       "client_id": APP_KEY,
       "client_secret": APP_SECRET,
       "redirect_uri": "http://localhost:8000"}
f = urlopen("https://api.dropboxapi.com/oauth2/token", urlencode(head).encode("ascii"))
token_json = eval(f.read().decode("utf-8"))
token = token_json["access_token"]

In [71]:
class DropboxRequest:
    def __init__(self,token):
        self.token = token
        
    @staticmethod
    def make_request(url, headers, data=None):
        """ Makes a request to Dropbox's API
        Args: 
            url, headers of call
        Returns: 
            a dict object
        """
        if(data == None):
            req = urllib.request.Request(url,None,headers)
        else:
            req = urllib.request.Request(url,json.dumps(data).encode(),headers)


        response = urllib.request.urlopen(req)
        return response.read().decode("utf-8")

    def list_folder(self,path):
        """ Gets a list of all files in a folder
        Args: 
            path: string path to the folder to look in
        Returns: a dict containing information about that folder
        """
        url = "https://api.dropboxapi.com/2/files/list_folder"
        headers = {"Authorization": "Bearer " + self.token,
                  "Content-Type": "application/json"}
        data = {"path": path}
        return DropboxRequest.make_request(url, headers,data=data)
    
    def download(self,path):
        url = "https://content.dropboxapi.com/2/files/download"
        headers = {
    "Authorization": "Bearer " + token,
    "Dropbox-API-Arg": '{"path":"/my-plugin.py"}'
        }
        return DropboxRequest.make_request(url, headers)
        
        

In [72]:
a = DropboxRequest(token)

In [74]:
a.list_folder("")

'{"entries": [{".tag": "folder", "name": "My_Files", "path_lower": "/my_files", "path_display": "/My_Files", "id": "id:MP9vQO9YC9AAAAAAAAAACw"}, {".tag": "file", "name": "Get Started with Dropbox.pdf", "path_lower": "/get started with dropbox.pdf", "path_display": "/Get Started with Dropbox.pdf", "id": "id:MP9vQO9YC9AAAAAAAAAABg", "client_modified": "2018-01-23T15:07:07Z", "server_modified": "2018-01-23T15:07:07Z", "rev": "186a84920", "size": 1102331, "content_hash": "f7ad488deb7d81790340ecd676fe6e47f0a6064fb99b982685b752d58611c1cb"}, {".tag": "file", "name": "Get Started with Dropbox Paper.url", "path_lower": "/get started with dropbox paper.url", "path_display": "/Get Started with Dropbox Paper.url", "id": "id:MP9vQO9YC9AAAAAAAAAABw", "client_modified": "2018-01-23T15:07:07Z", "server_modified": "2018-01-23T15:07:07Z", "rev": "286a84920", "size": 240, "content_hash": "f40c1228343d7e2a632281c986dbb7af3491b9b63ddfd0eb10fee2c913f6cfa7"}, {".tag": "file", "name": "my-plugin.py", "path_lo